In [4]:
import pandas as pd
import numpy as np

# 1. Load kedua dataset
# Ganti path sesuai dengan lokasi file Anda
df_dataset2 = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\dataset berita terkini 2\data_compressed.csv.gz', compression='gzip')
df_dataset3 = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\dataset berita terkini 3\politik_merge_compressed.csv.gz', compression='gzip')

# 2. Sesuaikan nama kolom agar konsisten
# Berdasarkan dataset yang Anda upload, dataset2 memiliki kolom 'content' dan 'title'
# Sedangkan dataset3 memiliki kolom 'Content' dan 'Judul'
df_dataset3 = df_dataset3.rename(columns={'Content': 'content', 'Judul': 'title', 'Waktu': 'date'})

# 3. Pilih hanya kolom yang dibutuhkan
df_dataset2 = df_dataset2[['content', 'title', 'date']]
df_dataset3 = df_dataset3[['content', 'title', 'date']]

# # 4. Hapus duplikat dan nilai kosong dari kedua dataset
# df_dataset2 = df_dataset2.drop_duplicates(subset=['content'])
# df_dataset2 = df_dataset2.dropna(subset=['content', 'title'])

# df_dataset3 = df_dataset3.drop_duplicates(subset=['content'])
# df_dataset3 = df_dataset3.dropna(subset=['content', 'title'])

# 5. Ambil 3500 sampel dari masing-masing dataset
sample_size = 3000
df_sample2 = df_dataset2.sample(n=sample_size, random_state=42)
df_sample3 = df_dataset3.sample(n=sample_size, random_state=42)

# 6. Gabungkan kedua sampel
df_combined = pd.concat([df_sample2, df_sample3], ignore_index=True)

# 7. Acak dataset gabungan untuk memastikan data tercampur dengan baik
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# 8. Simpan dataset gabungan ke file CSV
df_combined.to_csv('berita_terkini.csv', index=False)

# 9. Tampilkan informasi dataset
print(f"Jumlah data dari dataset2: {len(df_sample2)}")
print(f"Jumlah data dari dataset3: {len(df_sample3)}")
print(f"Jumlah data pada dataset gabungan: {len(df_combined)}")
print("\nContoh 5 baris data dari dataset gabungan:")
df_combined

Jumlah data dari dataset2: 3000
Jumlah data dari dataset3: 3000
Jumlah data pada dataset gabungan: 6000

Contoh 5 baris data dari dataset gabungan:


,content,title,date
0,"Presiden bertemu dengan timnas U-20 di GBK, Se...",Jokowi: Beberapa Pemain Timnas U-20 Ingin Kuli...,2023-04-01 10:36:35+00
1,Presiden Joko Widodo (Jokowi) melakukan kunjun...,"Kunker ke Lampung, Jokowi Akan Resmikan Bendun...","Senin, 26 Agu 2024 08:53 WIB"
2,"Menteri Koordinator Politik, Hukum, dan Keaman...","Mahfud Buat Satgas Tindak Lanjuti Rp 349 T, Di...",2023-04-10 05:28:58+00
3,JAKARTA - Cara tambah daya listrik hingga 5.50...,Tambah Daya Listrik hingga 5.500 VA dengan Bay...,2023-03-23 02:41:24+00
4,Sekretaris Jenderal PBNU Gus Saifullah Yusuf (...,"Sekjen PBNU: Hubungan Santri dan Polri Baik, S...","Selasa, 03 Sep 2024 12:43 WIB"
...,...,...,...
5995,"TEMPO.CO, Jakarta - Tim Pembela Prabowo-Gibran...","Hari Ini, Kubu Prabowo-Gibran Bakal Hadirkan 1...",4/4/2024
5996,Bakal calon Wakil Gubernur DKI Jakarta Rano Ka...,Rano Karno Ungkap Obrolan dengan Anies di Kant...,"Rabu, 28 Agu 2024 12:16 WIB"
5997,Lembaga Survei Indonesia (LSI) mengungkapkan b...,"Buah Kerja Keras Airin, Unggul di Semua Simula...","Kamis, 22 Agu 2024 11:45 WIB"
5998,"JAKARTA, KOMPAS.com - Jaksa Penuntut Umum (JP...","Jaksa Bocorkan ""Chat"" Gazalba dengan Perempuan...",8/8/2024


In [27]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from indonesian_number_normalizer import create_normalizer
from tqdm import tqdm

In [28]:
nltk.download('punkt')
nltk.download('stopwords')
normalizer = create_normalizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
df = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terkini.csv')
df

,content,title,date
0,"Presiden bertemu dengan timnas U-20 di GBK, Se...",Jokowi: Beberapa Pemain Timnas U-20 Ingin Kuli...,2023-04-01 10:36:35+00
1,Presiden Joko Widodo (Jokowi) melakukan kunjun...,"Kunker ke Lampung, Jokowi Akan Resmikan Bendun...","Senin, 26 Agu 2024 08:53 WIB"
2,"Menteri Koordinator Politik, Hukum, dan Keaman...","Mahfud Buat Satgas Tindak Lanjuti Rp 349 T, Di...",2023-04-10 05:28:58+00
3,JAKARTA - Cara tambah daya listrik hingga 5.50...,Tambah Daya Listrik hingga 5.500 VA dengan Bay...,2023-03-23 02:41:24+00
4,Sekretaris Jenderal PBNU Gus Saifullah Yusuf (...,"Sekjen PBNU: Hubungan Santri dan Polri Baik, S...","Selasa, 03 Sep 2024 12:43 WIB"
...,...,...,...
5995,"TEMPO.CO, Jakarta - Tim Pembela Prabowo-Gibran...","Hari Ini, Kubu Prabowo-Gibran Bakal Hadirkan 1...",4/4/2024
5996,Bakal calon Wakil Gubernur DKI Jakarta Rano Ka...,Rano Karno Ungkap Obrolan dengan Anies di Kant...,"Rabu, 28 Agu 2024 12:16 WIB"
5997,Lembaga Survei Indonesia (LSI) mengungkapkan b...,"Buah Kerja Keras Airin, Unggul di Semua Simula...","Kamis, 22 Agu 2024 11:45 WIB"
5998,"JAKARTA, KOMPAS.com - Jaksa Penuntut Umum (JP...","Jaksa Bocorkan ""Chat"" Gazalba dengan Perempuan...",8/8/2024


In [30]:
df = df.drop_duplicates(subset=['content']).reset_index(drop=True).copy(deep=True)
print(f"Dataset setelah dibersihkan memiliki {len(df)} baris")

Dataset setelah dibersihkan memiliki 5951 baris


PRE PROCESSING

In [31]:
def cleaning(teks):
    if not isinstance(teks, str):
        return ""
    teks = re.sub(r'<.*?>', ' ', teks)
    teks = re.sub(r'https?://\S+|www\.\S+', ' ', teks)
    teks = re.sub(r'ADVERTISEMENT.*?CONTENT', ' ', teks, flags=re.IGNORECASE | re.DOTALL)
    teks = re.sub(r'[^\w\s\d]', ' ', teks)
    teks = re.sub(r'\s+', ' ', teks).strip()
    return teks
df['cleaned_text'] = df['content'].fillna('').apply(cleaning)

In [32]:
def case_folding(teks):
    return teks.lower()
df['case_folded_text'] = df['cleaned_text'].apply(case_folding)

In [33]:
def normalisasi(teks):
    # Normalisasi angka (misalnya "100 juta" menjadi "seratus juta")
    try:
        # Gunakan normalize_text dari instance normalizer
        teks = normalizer.normalize_text(teks)
    except Exception as e:
        print(f"Error saat normalisasi angka: {e}")
        pass
    slang_dict = {
        'dgn': 'dengan',
        'tdk': 'tidak',
        'tsb': 'tersebut',
        'utk': 'untuk',
        'spy': 'supaya',
        'krn': 'karena',
        'jg': 'juga',
        'bs': 'bisa',
        'sdh': 'sudah',
        'blm': 'belum',
        'org': 'orang',
        'yg': 'yang',
        'sy': 'saya',
        'dlm': 'dalam',
        'pd': 'pada',
        'dr': 'dari',
        'kmrn': 'kemarin',
        'skrg': 'sekarang',
        'hrs': 'harus',
        'msk': 'masuk',
        'trs': 'terus',
        'tp': 'tapi',
        'kalo': 'kalau',
        'gak': 'tidak',
        'ga': 'tidak',
        'ngga': 'tidak',
        'gk': 'tidak',
        'thn': 'tahun',
        'bln': 'bulan',
        'sblm': 'sebelum',
        'stlh': 'setelah',
        'milyar': 'miliar',
        'trilliun': 'triliun',
        'jt': 'juta',
        'rb': 'ribu',
        '%': 'persen',
        'usd': 'dolar amerika',
        'rupiah': 'rupiah',
        'rp': 'rupiah',
        ',': 'koma',
    }
    
    # Buat pattern regex untuk seluruh kamus sekaligus (lebih efisien)
    pattern = r'\b(' + '|'.join(slang_dict.keys()) + r')\b'
    
    # Fungsi replacement yang menggunakan kamus
    def replace_match(match):
        return slang_dict[match.group(0)]
    
    # Ganti semua kata slang dalam satu kali proses
    return re.sub(pattern, replace_match, teks)
df['normalized_text'] = df['case_folded_text'].apply(normalisasi)

In [34]:
def tokenzing(teks):
    return word_tokenize(teks)
df['tokens'] = df['normalized_text'].apply(tokenzing)

In [35]:
indo_stopwords = set(stopwords.words('indonesian'))
tambahan_stopwords = {
    'ya', 'juga', 'dari', 'di', 'ke', 'pada', 'untuk', 'bagi', 'dan', 'atau', 
    'tapi', 'namun', 'dengan', 'secara', 'oleh', 'karena', 'sehingga', 'agar',
    'sebab', 'jika', 'bila', 'adalah', 'ini', 'itu', 'detik', 'kata', 'dalam',
    'saat', 'akan', 'tidak', 'yang', 'belum', 'sudah', 'telah', 'bisa', 'dapat', 
    'nya', 'pak', 'bu', 'hal', 'pun'
}
indo_stopwords.update(tambahan_stopwords)
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in indo_stopwords]
df['filtered_tokens'] = df['tokens'].apply(remove_stopwords)

Stemming:   0%|          | 6/5951 [4:12:28<4169:24:58, 2524.79s/it]


In [36]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming(tokens):
    return [stemmer.stem(word) for word in tokens]
df['stemmed_tokens'] = [stemming(tokens) for tokens in tqdm(df['filtered_tokens'], desc="Stemming")]
df['stemmed_text'] = df['stemmed_tokens'].apply(lambda x: ' '.join(x))

Stemming:   0%|          | 0/5951 [00:00<?, ?it/s]

Stemming: 100%|██████████| 5951/5951 [1:06:27<00:00,  1.49it/s]    


In [38]:
df['processed_content'] = df['stemmed_text']
display(df[['title', 'content', 'stemmed_text', 'date']].head())

,title,content,stemmed_text,date
0,Jokowi: Beberapa Pemain Timnas U-20 Ingin Kuli...,"Presiden bertemu dengan timnas U-20 di GBK, Se...",presiden temu timnas u puluh gbk senayan sabtu...,2023-04-01 10:36:35+00
1,"Kunker ke Lampung, Jokowi Akan Resmikan Bendun...",Presiden Joko Widodo (Jokowi) melakukan kunjun...,presiden joko widodo jokowi kunjung kerja lamp...,"Senin, 26 Agu 2024 08:53 WIB"
2,"Mahfud Buat Satgas Tindak Lanjuti Rp 349 T, Di...","Menteri Koordinator Politik, Hukum, dan Keaman...",menteri koordinator politik hukum aman menko p...,2023-04-10 05:28:58+00
3,Tambah Daya Listrik hingga 5.500 VA dengan Bay...,JAKARTA - Cara tambah daya listrik hingga 5.50...,jakarta daya listrik ratus va bayar rpdua ratu...,2023-03-23 02:41:24+00
4,"Sekjen PBNU: Hubungan Santri dan Polri Baik, S...",Sekretaris Jenderal PBNU Gus Saifullah Yusuf (...,sekretaris jenderal pbnu gus saifullah yusuf g...,"Selasa, 03 Sep 2024 12:43 WIB"


In [47]:
preprocessing_new_file = r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terkini_preprocessing.csv'
preprocessing_new_columns = ['title', 'content', 'stemmed_text', 'date']
df[preprocessing_new_columns].to_csv(preprocessing_new_file, index=False)
print(f"Hasil preprocessing berita terkini disimpan ke {preprocessing_new_file}")

Hasil preprocessing berita terkini disimpan ke D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terkini_preprocessing.csv


PENGELOMPOKAN FAKTOR

In [42]:
# Kata kunci untuk setiap kategori
kata_kunci = {
    'suku_bunga': [
        'suku bunga', 'interest rate', 'bi rate', 'bunga', 'bi 7drrr', 'rate', 
        'interest', 'bunga pinjaman', 'kredit', 'pinjaman', 'kpr', 'deposito', 
        'tabungan', 'kebijakan moneter', 'pemangkasan bunga', 'kenaikan bunga', 
        'penurunan bunga', 'bunga acuan', 'fed rate', 'the fed', 'bank sentral', 
        'bank indonesia', 'inflasi', 'yield', 'obligasi', 'federal reserve',
        'persen', 'persentase', 'basis poin', 'bps', 'naik', 'turun', 'tetap',
        'tingkat bunga', 'acuan moneter', 'kebijakan moneter', 'surat utang'
    ],
    'impor': [
        'impor', 'import', 'bea masuk', 'tarif impor', 'pajak impor', 'barang impor', 
        'komoditas impor', 'produk impor', 'ketergantungan impor', 'pembatasan impor',
        'larangan impor', 'kuota impor', 'izin impor', 'nilai impor', 'volume impor', 
        'impor beras', 'impor gula', 'impor daging', 'impor bbm', 'impor gandum', 
        'impor kedelai', 'impor bawang', 'impor garam', 'impor jagung',
        'importir', 'barang masuk', 'masuk negara', 'kebutuhan dalam negeri', 
        'ketergantungan', 'kuota', 'belanja luar negeri', 'pembelian luar negeri',
        'pelabuhan', 'bongkar muat', 'container', 'kontainer', 'harmonisasi'
    ],
    'ekspor': [
        'ekspor', 'export', 'komoditas ekspor', 'produk ekspor', 'nilai ekspor', 
        'volume ekspor', 'pasar ekspor', 'tujuan ekspor', 'pertumbuhan ekspor', 
        'penurunan ekspor', 'kenaikan ekspor', 'ekspor sawit', 'ekspor batu bara', 
        'ekspor nikel', 'ekspor karet', 'ekspor tekstil', 'ekspor manufaktur', 
        'ekspor pertanian', 'ekspor perikanan', 'ekspor kayu', 'eksportir', 
        'perdagangan internasional', 'neraca perdagangan', 'surplus perdagangan', 
        'defisit perdagangan','barang keluar', 'pasar global', 'pasar dunia', 'pengiriman barang',
        'devisa', 'penghasil devisa', 'penerimaan negara', 'komoditas unggulan',
        'nilai tambah', 'pengapalan', 'negara tujuan', 'kepabeanan', 'hilirisasi'
    ]
}

In [43]:
stemmed_kata_kunci = {}
for kategori, keywords in kata_kunci.items():
    stemmed_keywords = []
    for keyword in keywords:
        if ' ' in keyword:  # Kata majemuk
            stemmed_parts = [stemmer.stem(word) for word in keyword.split()]
            stemmed_keywords.append(' '.join(stemmed_parts))
        else:  # Kata tunggal
            stemmed_keywords.append(stemmer.stem(keyword))
    stemmed_kata_kunci[kategori] = stemmed_keywords

# Fungsi untuk memeriksa apakah teks mengandung kata kunci dari kategori tertentu
# Perbaikan: Menggunakan regex word boundary untuk pencocokan kata yang lebih akurat
def cek_kategori(teks, kategori_keywords):
    matches = []
    for keyword in kategori_keywords:
        if ' ' in keyword:  # Kata majemuk - cek sebagai frasa
            if keyword in teks:
                matches.append(keyword)
        else:  # Kata tunggal - gunakan regex dengan word boundary
            pattern = r'\b' + re.escape(keyword) + r'\b'
            if re.search(pattern, teks):
                matches.append(keyword) 
    return matches

# Kategorisasi berita dengan pencatatan kata kunci yang cocok
for kategori, keywords in stemmed_kata_kunci.items():
    df[f"{kategori}_keywords"] = df['stemmed_text'].apply(lambda x: cek_kategori(x, keywords))
    df[kategori] = df[f"{kategori}_keywords"].apply(lambda x: len(x) > 0)

In [44]:
#Membuat kolom faktor
def get_faktor(row):
    faktor = []
    for kategori in stemmed_kata_kunci.keys():
        if row[kategori]:
            faktor.append(kategori)
    return faktor
df['faktor'] = df.apply(get_faktor, axis=1)

# Menambahkan kolom yang mencatat kata kunci spesifik yang ditemukan
def get_keyword_details(row):
    details = {}
    for kategori in stemmed_kata_kunci.keys():
        if row[kategori] and len(row[f"{kategori}_keywords"]) > 0:
            details[kategori] = row[f"{kategori}_keywords"]
    return details
df['keyword_details'] = df.apply(get_keyword_details, axis=1)

print("\nJumlah berita per kategori:")
for kategori in stemmed_kata_kunci.keys():
    jumlah = df[kategori].sum()
    print(f"{kategori}: {jumlah} berita")


Jumlah berita per kategori:
suku_bunga: 2111 berita
impor: 521 berita
ekspor: 383 berita


In [ ]:
df['jumlah_kategori'] = df[list(stemmed_kata_kunci.keys())].sum(axis=1)
print(f"\nBerita yang tidak masuk kategori: {len(df[df['jumlah_kategori'] == 0])}")
print(f"Berita yang masuk 1 kategori: {len(df[df['jumlah_kategori'] == 1])}")
print(f"Berita yang masuk 2 kategori: {len(df[df['jumlah_kategori'] == 2])}")
print(f"Berita yang masuk 3 kategori: {len(df[df['jumlah_kategori'] == 3])}")


Berita yang tidak masuk kategori: 3414
Berita yang masuk 1 kategori: 2123
Berita yang masuk 2 kategori: 350
Berita yang masuk 3 kategori: 64


In [46]:
# Menghapus berita yang tidak masuk ke kategori apapun
df_filtered = df[df['jumlah_kategori'] > 0].copy()
print(f"Jumlah berita setelah menghapus yang tidak berkategori: {len(df_filtered)}")

Jumlah berita setelah menghapus yang tidak berkategori: 2537


In [ ]:
# # Menyimpan hasil kategori ke file CSV baru
# output_file = r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terkini_berkategori.csv'
# # Mengonversi kolom faktor dan keyword_details (dict/list) menjadi string agar bisa disimpan di CSV
# df_filtered['faktor_str'] = df_filtered['faktor'].apply(lambda x: str(x))
# df_filtered['keyword_details_str'] = df_filtered['keyword_details'].apply(lambda x: str(x))
# df_output = df_filtered[['title', 'content', 'date', 'faktor_str', 'jumlah_kategori', 'keyword_details_str']]
# df_output.to_csv(output_file, index=False)
# print(f"Hasil pengelompokan telah disimpan ke {output_file}")

Hasil pengelompokan telah disimpan ke D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terkini_berkategori.csv


In [55]:
preprocessing_file = (r'D:\SKRIPSI\skripsi_2025\fix_dataset\fix_berita_terkini.csv')
preprocessing_columns = ['title', 'content','stemmed_text', 'faktor_str', 'date']
df_filtered[preprocessing_columns].to_csv(preprocessing_file, index=False)
print(f"Hasil preprocessing (hanya berita berkategori) disimpan ke {preprocessing_file}")

Hasil preprocessing (hanya berita berkategori) disimpan ke D:\SKRIPSI\skripsi_2025\fix_dataset\fix_berita_terkini.csv


In [48]:
print("\nSampel berita dari setiap kategori:")
for kategori in stemmed_kata_kunci.keys():
    print(f"\n=== SAMPEL BERITA {kategori.upper()} ===")
    if df[df[kategori]].shape[0] > 0:
        sampel = df[df[kategori]].sample(min(3, df[df[kategori]].shape[0]))
        for idx, row in sampel.iterrows():
            print(f"Judul: {row['title']}")
            print(f"Tanggal: {row['date']}")
            print(f"Faktor: {row['faktor']}")
            print(f"Kata kunci yang ditemukan: {row['keyword_details']}")
            print("---")
    else:
        print("Tidak ada berita dalam kategori ini.")



Sampel berita dari setiap kategori:

=== SAMPEL BERITA SUKU_BUNGA ===
Judul: Terpilih Jadi Ketum NasDem Lagi, Surya Paloh Sebut Wajar Sebagai Pendiri Partai
Tanggal: 8/27/2024
Faktor: ['suku_bunga']
Kata kunci yang ditemukan: {'suku_bunga': ['tetap']}
---
Judul: Bukan Ikan Didapat, Nelayan Ini Malah Peroleh Sabu 15 Kg di Laut
Tanggal: 2023-04-04 07:04:18+00
Faktor: ['suku_bunga', 'ekspor']
Kata kunci yang ditemukan: {'suku_bunga': ['naik'], 'ekspor': ['kapal']}
---
Judul: Pengusungan Ridwan Kamil di Jakarta Dinilai Hasil Kompromi Pembagian Wilayah KIM
Tanggal: 10/8/2024
Faktor: ['suku_bunga']
Kata kunci yang ditemukan: {'suku_bunga': ['persen']}
---

=== SAMPEL BERITA IMPOR ===
Judul: KPU Bahas Aturan Pencalonan, Pastikan Caleg Terpilih Harus Mundur Jika Maju di Pilkada 2024
Tanggal: 5/20/2024
Faktor: ['impor']
Kata kunci yang ditemukan: {'impor': ['harmonisasi']}
---
Judul: Ibu-Ibu! Harga Daging Sapi Bakal Turun, Kapan?
Tanggal: 2023-03-18 10:17:47+00
Faktor: ['suku_bunga', 'impor']
